In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras.layers import  Dropout,Conv2D,Flatten,MaxPool2D
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,

In [ ]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'cry no_cry'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'ba_cry/{g}'):
        songname = f'ba_cry/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

In [ ]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'cry no_cry'.split()
for g in genres:
    for filename in os.listdir(f'ba_cry/{g}'):
        songname = f'ba_cry/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=7)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [5]:
data = pd.read_csv('dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
# X =np.array(data.iloc[:, :-1], dtype = float)
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<Figure size 432x288 with 0 Axes>

In [9]:
model = Sequential()
# model.add(Conv2D(16, (3, 3), activation='relu', strides=(1, 1),padding='same', input_shape=X_train.shape[1]))
# model.add(Conv2D(32, (3, 3), activation='relu', strides=(1, 1),padding='same')) 
# model.add(Conv2D(64, (3, 3), activation='relu', strides=(1, 1),padding='same'))
# model.add(MaxPool2D ((2, 2)))
# model.add(Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [10]:
model.fit(X_train,y_train,shuffle=True,epochs=50,batch_size=128)

Epoch 1/50
1/1 [==============================] - ETA: 0s - loss: 0.6631 - accuracy: 0.54 - 0s 5ms/step - loss: 0.6631 - accuracy: 0.5408
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 0.7008 - accuracy: 0.62 - 0s 8ms/step - loss: 0.7008 - accuracy: 0.6224
Epoch 3/50
1/1 [==============================] - ETA: 0s - loss: 0.5543 - accuracy: 0.72 - 0s 9ms/step - loss: 0.5543 - accuracy: 0.7245
Epoch 4/50
1/1 [==============================] - ETA: 0s - loss: 0.5698 - accuracy: 0.73 - 0s 6ms/step - loss: 0.5698 - accuracy: 0.7347
Epoch 5/50
1/1 [==============================] - ETA: 0s - loss: 0.5464 - accuracy: 0.69 - 0s 8ms/step - loss: 0.5464 - accuracy: 0.6939
Epoch 6/50
1/1 [==============================] - ETA: 0s - loss: 0.5094 - accuracy: 0.77 - 0s 6ms/step - loss: 0.5094 - accuracy: 0.7755
Epoch 7/50
1/1 [==============================] - ETA: 0s - loss: 0.5021 - accuracy: 0.74 - 0s 5ms/step - loss: 0.5021 - accuracy: 0.7449
Epoch 8/50
1/1 [==================

In [13]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


* recording
* done recording


<Figure size 432x288 with 0 Axes>

In [14]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
pathlib.Path(f'live_data/').mkdir(parents=True, exist_ok=True)
# for filename in os.listdir(f'b_cry/{g}'):
songname = f'output.wav'
y, sr = librosa.load(songname, mono=True, duration=7)
plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
plt.axis('off');
plt.savefig(f'live_data/output.png')
plt.clf()

<Figure size 576x576 with 0 Axes>

In [15]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [16]:
file = open('predict.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
# genres = 'discomfort belly_pain burping hungry'.split()
# for g in genres:
#     for filename in os.listdir(f'b_cry/{g}'):
# songname = f'b_cry/{g}/{filename}'
songname = f'output.wav'
y, sr = librosa.load(songname, mono=True, duration=5)
rmse = librosa.feature.rms(y=y)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
to_append = f'output.wav {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'   
for e in mfcc:
    to_append += f' {np.mean(e)}'
# to_append += f' {model.predict(live_data.iloc[0])}'
#to_append += f' live_data_genre'
file = open('predict.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())
live_data = pd.read_csv('predict.csv')
live_data = live_data.drop(['filename'],axis=1)


In [17]:
op=list(to_append.split())
print(op.pop(0))

output.wav


In [21]:
#print(live_data.iloc[0][0:26])

print(op)
Xnew = scaler.transform([op])
print(Xnew)
ynew = model.predict_classes(Xnew)
print("Predicted=%s" % (ynew))
if ynew==0:
    print('Cry')
else:
    print('No cry')
#live_data = data.drop(['filename'],axis=1)
#print(model.predict(live_data.iloc[:, -1]))
# print(type(live_data.iloc[0]))
#print(model.predict_classes(live_data.iloc[0]))
# print(live_data.iloc[0])

['0.40205520391464233', '0.019040800631046295', '2029.287070207813', '2492.9662724210903', '4587.3199462890625', '0.05556685836226852', '-352.869140625', '100.57908630371094', '10.353883743286133', '15.111245155334473', '6.137244701385498', '1.810538411140442', '-7.785067081451416', '8.381434440612793', '-5.35196590423584', '16.317611694335938', '1.39741051197052', '4.00768518447876', '6.031613349914551', '-3.389061212539673', '4.086082935333252', '9.323531150817871', '-6.910163402557373', '7.099420547485352', '-0.2120068520307541', '4.200275897979736']
[[-1.26592688e-01 -7.14185881e-01 -3.06876729e-01  5.27057018e-01
   2.34328967e-04 -8.91113351e-01 -3.12259516e-01  1.02326634e+00
   1.01659183e+00  7.33190136e-01  8.83629830e-01 -1.99488609e-01
  -1.56417889e-01  3.80499371e-01 -5.88635751e-01  1.71146650e+00
   2.20608464e-01  1.86673715e-01  9.81535019e-01 -8.71235627e-01
   5.56513848e-01  7.10579287e-01 -1.18968169e+00  6.48530049e-01
   1.99159960e-01  2.23600507e-01]]
Predicte

<Figure size 432x288 with 0 Axes>

In [11]:
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 100.00%
Testing Accuracy: 92.00%


In [6]:
import pyaudio
import wave
import pathlib
import threading
import csv
import time

# t1 = threading.Thread(target=print_square, args=(10,))
# t2 = threading.Thread(target=print_cube, args=(10,))

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 6
WAVE_OUTPUT_FILENAME = "output.wav"

stop = 0


p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")


def predict(name):
    #Doing prediction
    cmap = plt.get_cmap('inferno')
    plt.figure(figsize=(8,8))
    pathlib.Path(f'live_data/').mkdir(parents=True, exist_ok=True)
    # for filename in os.listdir(f'b_cry/{g}'):
    songname = name+'.wav'
    y, sr = librosa.load(songname, mono=True, duration=7)
    plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
    plt.axis('off');
    plt.savefig(f'live_data/{name}.png')
    plt.clf()

    header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    for i in range(1, 21):
        header += f' mfcc{i}'
    header += ' label'
    header = header.split()

    file = open(f'{name}.csv', 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
    # genres = 'discomfort belly_pain burping hungry'.split()
    # for g in genres:
    #     for filename in os.listdir(f'b_cry/{g}'):
    # songname = f'b_cry/{g}/{filename}'
#     songname = f'output.wav'
    y, sr = librosa.load(songname, mono=True, duration=5)
    rmse = librosa.feature.rms(y=y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{name}.wav {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'   
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    # to_append += f' {model.predict(live_data.iloc[0])}'
    #to_append += f' live_data_genre'
    file = open(f'{name}.csv', 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())
    live_data = pd.read_csv(f'{name}.csv')
    live_data = live_data.drop(['filename'],axis=1)

    #print(live_data.iloc[0][0:26])
    op=list(to_append.split())
    op.pop(0)
#     print(Xnew)
    Xnew = scaler.transform([op])
    ynew = model.predict_classes(Xnew)

    print("Predicted=%s" % (ynew))
    if ynew==0:
        print('Cry')
    else:
        print('No cry')


# def record_audio():
for j in range(0,4):
    frames = []
    songname = "output"+str(j)
    WAVE_OUTPUT_FILENAME = "output"+str(j)+".wav"
    print(WAVE_OUTPUT_FILENAME)
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    
    print("Predicting")
#     for i in range(0,5):
#         print(str(i*100))
#         time.sleep(0.5)
    t2 = threading.Thread(target=predict, args=(songname,))
    t2.start()
    
# if __name__ == "__main__":
    # creating thread
# t1 = threading.Thread(target=record_audio, args=())
# t2 = threading.Thread(target=predict, args=())

# while(stop==0):
# t1.start()
# t2.start()
# t1.join()
# t2.join()
# both threads completely executed
print("Done!")

* recording
output0.wav
Predicting
output1.wav


Exception in thread Thread-14:
Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\User\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-5ac50552d28a>", line 84, in predict
    Xnew = scaler.transform([op])
  File "C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py", line 763, in transform
    X -= self.mean_
ValueError: operands could not be broadcast together with shapes (1,26) (40,) (1,26) 



Predicting
output2.wav


Exception in thread Thread-15:
Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\User\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-5ac50552d28a>", line 84, in predict
    Xnew = scaler.transform([op])
  File "C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py", line 763, in transform
    X -= self.mean_
ValueError: operands could not be broadcast together with shapes (1,26) (40,) (1,26) 



Predicting
output3.wav


...

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

<Figure size 576x576 with 0 Axes>

Exception in thread Thread-17:
Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\User\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-5ac50552d28a>", line 84, in predict
    Xnew = scaler.transform([op])
  File "C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py", line 763, in transform
    X -= self.mean_
ValueError: operands could not be broadcast together with shapes (1,26) (40,) (1,26) 



In [ ]:
# from keras.models import load_model
# model1 = load_model('ff_nn_v1_Ac85.h5')

In [ ]:
# from keras.models import load_model
# model.save("ff_nn_v2_Ac92.h5")

In [ ]:
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.io import wavfile # get the api
fs, data = wavfile.read('output3.wav') # load the data
a = data.T[0] # this is a two channel soundtrack, I get the first track
b=[(ele/2**8.)*2-1 for ele in a] # this is 8-bit track, b is now normalized on [-1,1)
c = fft(b) # calculate fourier transform (complex numbers list)
d = len(c)/2  # you only need half of the fft list (real signal symmetry)
plt.plot(abs(c[0:(5-1)]),'r') 
plt.show()